In [1]:
import torch
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib
DEVICE = 'cuda:0'
from qlib.utils.incoherence_preprocessing.incoherence_process_functions import incoherence_process, incoherence_preprocess, incoherence_preprocess_lukashevich, incoherence_process_lukashevich

In [2]:
model_name='Llama2-7b-hf'
DTYPE = torch.float16
model = qlib.load_model(model_name=model_name, torch_dtype=DTYPE)

module = model.get_decoder().layers[0].self_attn.q_proj.to(DEVICE)
#module = model.get_decoder().layers[10].mlp.down_proj.to(DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
w_params = qlib.TrellisQuantizerParams(
	T=256,
	L=16,
	K=2,
	V=2,
	decode_mode='LowBitSym',
	tlut_bits=10,
)

qmodule = qlib.TrellisLinear(
	weight_shape=module.weight.shape, 
	weight_scales_group_size=128,
	incoh_proc_mode='lukashevich',
	init_device='cuda:0',
	input_quantizer_params=None,
	weight_quantizer_params=w_params
)

In [4]:
qmodule = qmodule.wrap_module(module, verbose=True)

100%|██████████| 256/256 [00:19<00:00, 12.99it/s]


error (w-wq)^2: 0.070


In [5]:
qmodule.SU #1

Parameter containing:
tensor([-0.1096,  0.1096,  0.1096,  ...,  0.1096,  0.1096,  0.1096],
       device='cuda:0', requires_grad=True)

In [6]:
qmodule.weight_scales #0.0038, 0.0060

Parameter containing:
tensor([[0.0351],
        [0.0351],
        [0.0351],
        ...,
        [0.0552],
        [0.0552],
        [0.0552]], device='cuda:0', dtype=torch.float16, requires_grad=True)

In [7]:
module.weight

Parameter containing:
tensor([[-0.0062, -0.0148, -0.0022,  ...,  0.0045,  0.0017, -0.0036],
        [ 0.0142, -0.0043,  0.0028,  ..., -0.0093, -0.0114,  0.0076],
        [-0.0146,  0.0126,  0.0005,  ...,  0.0063,  0.0188, -0.0031],
        ...,
        [ 0.0013,  0.0109, -0.0003,  ...,  0.0098, -0.0298,  0.0097],
        [ 0.0256,  0.0102,  0.0032,  ..., -0.0334, -0.0156, -0.0123],
        [-0.0134, -0.0066,  0.0018,  ...,  0.0181,  0.0166, -0.0082]],
       device='cuda:0', dtype=torch.float16, requires_grad=True)

In [8]:
incoherence_process_lukashevich(qmodule.weight, qmodule.SU.half())

tensor([[-0.0091, -0.0117, -0.0035,  ...,  0.0056,  0.0011,  0.0005],
        [ 0.0166,  0.0061,  0.0003,  ..., -0.0093, -0.0082,  0.0106],
        [-0.0178,  0.0137, -0.0036,  ..., -0.0067,  0.0144, -0.0065],
        ...,
        [ 0.0008,  0.0075,  0.0015,  ...,  0.0123, -0.0263,  0.0105],
        [ 0.0194,  0.0103,  0.0119,  ..., -0.0352, -0.0063, -0.0071],
        [-0.0072, -0.0068,  0.0026,  ...,  0.0115,  0.0164, -0.0096]],
       device='cuda:0', dtype=torch.float16, grad_fn=<MulBackward0>)

In [9]:
raise

RuntimeError: No active exception to reraise

In [4]:
# m = 1024
# n = 1024

# w = torch.randn(m, n, device=DEVICE)

params = qlib.TrellisQuantizerParams(
	T=256,
	L=16,
	K=2,
	V=2,
	decode_mode='LowBitSym',
	tlut_bits=10,
)

quantizer = qlib.TrellisQuantizer(
	params=params
).to(DEVICE)

reco, states = quantizer.quantize(w)
packed = quantizer.pack_trellis(states)

reco_ref = quantizer.reconstruct_weight(packed, w.shape)
err_ref  = torch.mean((reco_ref * quantizer.codebook_scale - w)**2, dim=-1)
print(f"error (unpack): {err_ref.mean():.3f} ± {err_ref.std():.3f}")


# reco_fast = quantizer.reconstruct_weight_fast(packed, w.shape)
# err_fast = torch.mean((reco_fast * quantizer.codebook_scale - w)**2, dim=-1)
# print(f"error (fast)  : {err_fast.mean():.3f} ± {err_fast.std():.3f}")

# 0.128

100%|██████████| 256/256 [00:19<00:00, 13.04it/s]


error (unpack): 0.074 ± 0.035


In [8]:
#Wq_scaled = incoherence_process_lukashevich(reco_ref * quantizer.codebook_scale, SU)
Wq_scaled = incoherence_process(reco_ref * quantizer.codebook_scale, SU, SV)
err = torch.mean((Wq_scaled - w_orig / w_orig_scales)**2, dim=-1)
print(f"error (fast)  : {err.mean():.3f} ± {err.std():.3f}")

error (fast)  : 0.074 ± 0.003


In [6]:
raise

RuntimeError: No active exception to reraise

### Without Scaling per group

In [3]:
fp_model = qlib.load_model('Llama2-7b-hf', torch_dtype=torch.float16)
W = fp_model.get_submodule('model.layers.0.self_attn.q_proj').weight.data.to(DEVICE)
Wr, SU, SV = incoherence_preprocess(W)

Wr = Wr.reshape(-1, 256)
scales = Wr.std()
Wr_scaled = Wr / scales
Wr_scaled = Wr_scaled.reshape_as(W)


quantizer = qlib.trellis_quantizer(
	L=16,
	K=2,
	V=2,
	T=256,
	decode_mode="LowBitSym", 
	tlut_bits=10
).to(DEVICE)

Wr_scaled_q, states = quantizer.quantize(Wr_scaled)

Wr_scaled_q = Wr_scaled_q.reshape(-1, 256)
Wr_q = Wr_scaled_q * scales
Wr_q = Wr_q.reshape_as(W)

W_q = incoherence_process(Wr_q, SU, SV)

err  = torch.mean(((W_q - W)**2).reshape(-1, 256), dim=-1)
print(f"error: {err.mean():.3e} ± {err.std():.3e}")

#error: 1.512e-05 ± 2.830e-06

#error: 5.038e-05 ± 5.205e-06

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 256/256 [00:20<00:00, 12.79it/s]


error: 1.377e-05 ± 2.306e-06


### With Scaling per group

In [4]:
fp_model = qlib.load_model('Llama2-7b-hf', torch_dtype=torch.float16)
W = fp_model.get_submodule('model.layers.0.self_attn.q_proj').weight.data.to(DEVICE)
Wr, SU, SV = incoherence_preprocess(W)

Wr = Wr.reshape(-1, 256)
scales = Wr.std(dim=-1, keepdim=True)
Wr_scaled = Wr / scales
Wr_scaled = Wr_scaled.reshape_as(W)

quantizer = qlib.trellis_quantizer(
	L=16,
	K=2,
	V=2,
	T=256,
	decode_mode="LowBitSym", 
	tlut_bits=10
).to(DEVICE)

Wr_scaled_q, states = quantizer.quantize(Wr_scaled)
Wr_scaled_q = Wr_scaled_q.reshape(-1, 256)
Wr_q = Wr_scaled_q * scales
Wr_q = Wr_q.reshape_as(W)
W_q = incoherence_process(Wr_q, SU, SV)

err  = torch.mean(((W_q - W)**2).reshape(-1, 256), dim=-1)
print(f"error: {err.mean():.3e} ± {err.std():.3e}")

# error: 5.013e-05 ± 5.153e-06

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 256/256 [00:20<00:00, 12.79it/s]


error: 1.311e-05 ± 1.842e-06


In [5]:
# %%timeit
# reco = quantizer.reconstruct_weight(packed, w.shape)

In [6]:
# %%timeit
# reco_fast = quantizer.reconstruct_weight_fast(packed, w.shape)